# Mid Term

In [23]:
import yahoo_fin.stock_info as si
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#import pyfolio as pf
from IPython.display import display

In [20]:
import requests
import bs4 as bs

def get_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
    tickers = sorted([x[:-1].lstrip().rstrip() for x in tickers])
    return tickers

In [44]:
def getsummary(ticker):
    # table = si.get_quote_table(ticker, dict_result = False)
    table_info = {
        "key": [],
        "value": [],
    }

    stock = yf.Ticker(ticker)

    table_info["key"].append("Previous Close")
    table_info["value"].append(stock.info["previousClose"])
    table_info["key"].append("Open")
    table_info["value"].append(stock.info["open"])
    table_info["key"].append("Bid")
    table_info["value"].append(stock.info["bid"])
    table_info["key"].append("Ask")
    table_info["value"].append(stock.info["ask"])


    table = pd.DataFrame(table_info)
    return table 

# getsummary('MSFT')

In [49]:
def getstockdata(ticker):
    stockdata = yf.download(ticker, period = 'MAX')
    return stockdata

# getstockdata("AAPL")

In [88]:
def getanalysis(ticker):
    analysts_site = "https://finance.yahoo.com/quote/"+ ticker +"/analysis"
    headers = {'User-agent': 'Mozilla/5.0'}
    tables = pd.read_html(requests.get(analysts_site, headers=headers).text)
    table_names = ["Earnings Estimate", 
                   "Revenue Estimate", 
                   "Earnings History",
                   "EPS Trend", 
                   "EPS Revisions",
                   "Growth Estimates"]
    table_mapper = {key : val for key , val in zip(table_names , tables[:len(table_names)])}
    analysis_dict = table_mapper
    return analysis_dict.items()

## Tab 1

In [24]:
tickers_list = ['-'] + get_sp500_tickers() # si.tickers_sp500()
display(tickers_list[:5])


['-', 'A', 'AAL', 'AAPL', 'ABBV']

In [45]:
ticker = "AAPL"

In [46]:
# Lấy thông tin cổ phiếu
getsummary(ticker)

,key,value
0,Previous Close,192.25
1,Open,192.98
2,Bid,194.08
3,Ask,194.28


In [51]:
# Vẽ đồ thị
chartdata = getstockdata(ticker) 
fig = px.area(chartdata, chartdata.index, chartdata['Close'])

[*********************100%%**********************]  1 of 1 completed


In [52]:
fig.update_xaxes(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1M", step="month", stepmode="backward"),
                    dict(count=3, label="3M", step="month", stepmode="backward"),
                    dict(count=6, label="6M", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1Y", step="year", stepmode="backward"),
                    dict(count=3, label="3Y", step="year", stepmode="backward"),
                    dict(count=5, label="5Y", step="year", stepmode="backward"),
                    dict(label = "MAX", step="all")
                ])
            )
        )

## Tab 5

In [100]:
stock = yf.Ticker("AAPL")

In [ ]:
ticker = "AAPL"

In [97]:
analysis = getanalysis(ticker)
print(len(analysis))
for i in range(len(analysis)):
    name = list(analysis)[i][0]
    df = pd.DataFrame(list(analysis)[i][1])
    display(name)
    display(df)

6


'Earnings Estimate'

,CURRENCY IN USD,Current Qtr. (Jun 2024),Next Qtr. (Sep 2024),Current Year (2024),Next Year (2025)
0,No. of Analysts,27.00,26.00,40.00,40.00
1,Avg. Estimate,1.22,1.41,6.07,6.66
2,Low Estimate,1.17,1.33,5.92,5.89
3,High Estimate,1.25,1.50,6.37,7.28
4,Year Ago EPS,1.16,1.34,5.65,6.07


'Revenue Estimate'

,CURRENCY IN USD,Current Qtr. (Jun 2024),Next Qtr. (Sep 2024),Current Year (2024),Next Year (2025)
0,No. of Analysts,25,25,38,38
1,Avg. Estimate,77.15B,85.06B,355.93B,378.81B
2,Low Estimate,75.48B,81.21B,348.38B,351.9B
3,High Estimate,78.63B,90.68B,362.63B,401.56B
4,Year Ago Sales,75.33B,82.43B,353.01B,355.93B
5,Sales Growth (year/est),2.40%,3.20%,0.80%,6.40%


...

,CURRENCY IN USD,Current Qtr. (Jun 2024),Next Qtr. (Sep 2024),Current Year (2024),Next Year (2025)
0,Current Estimate,1.22,1.41,6.07,6.66
1,7 Days Ago,1.32,1.53,6.59,7.23
2,30 Days Ago,1.22,1.43,6.09,6.64
3,60 Days Ago,1.32,1.55,6.56,7.16
4,90 Days Ago,1.32,1.55,6.56,7.17


'EPS Revisions'

,CURRENCY IN USD,Current Qtr. (Jun 2024),Next Qtr. (Sep 2024),Current Year (2024),Next Year (2025)
0,Up Last 7 Days,--,--,--,--
1,Up Last 30 Days,10,7,20,18
2,Down Last 7 Days,--,--,--,--
3,Down Last 30 Days,--,--,--,--


'Growth Estimates'

,CURRENCY IN USD,AAPL,Industry,Sector,S&P 500
0,Current Qtr.,5.20%,--,--,8.90%
1,Next Qtr.,5.20%,--,--,11.40%
2,Current Year,7.40%,--,--,5.80%
3,Next Year,9.70%,--,--,12.60%
4,Next 5 Years (per annum),9.67%,--,--,11.45%
5,Past 5 Years (per annum),20.15%,--,--,--


In [65]:
analysis = getanalysis(ticker)
for i in range(6):
    df = pd.DataFrame(list(analysis)[i][1])
    display(df)

,CURRENCY IN USD,AAPL,Industry,Sector,S&P 500
0,Current Qtr.,5.20%,--,--,8.90%
1,Next Qtr.,5.20%,--,--,11.40%
2,Current Year,7.40%,--,--,5.80%
3,Next Year,9.70%,--,--,12.60%
4,Next 5 Years (per annum),9.67%,--,--,11.45%
5,Past 5 Years (per annum),20.15%,--,--,--


,0,1,2
0,Maintains,Tigress Financial: Strong Buy to Strong Buy,5/30/2024
1,Maintains,B of A Securities: Buy to Buy,5/29/2024
2,Maintains,Wedbush: Outperform to Outperform,5/24/2024
3,Reiterates,Wedbush: Outperform to Outperform,5/13/2024
4,Maintains,Canaccord Genuity: Buy to Buy,5/3/2024
5,Upgrade,Itau BBA: Underperform to Market Perform,5/3/2024


IndexError: list index out of range

# End